### Building a neural network in PyTorch

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 kernel convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x)) # its like a flatten of tensorflow
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
params = list(net.parameters())  # list out all parameters
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [4]:
input = torch.randn(1, 1, 32, 32) # (batch, C, W, H)
out = net(input)
print(out)

tensor([[-0.0057,  0.0673, -0.1442,  0.1006,  0.0877,  0.0826, -0.0611,  0.0510,
         -0.0587, -0.0060]], grad_fn=<AddmmBackward>)


In [5]:
# Zero the gradient buffers of all parameters and backprops with random gradients:

In [6]:
net.zero_grad() # Sets gradients of all model parameters to zero.

In [7]:
out.backward(torch.randn(1, 10))

In [8]:
# loss function

In [9]:
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()
output = net(input)

In [10]:
loss = criterion(output, target)

In [11]:
print(loss)

tensor(1.2468, grad_fn=<MseLossBackward>)


In [12]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [13]:
# Backprop

In [14]:
net.zero_grad()
print('conv1.weight.grad before backward') # shape = torch.Size([6, 1, 3, 3])
print(net.conv1.weight.grad)

print('conv1.bias.grad before backward') # shape = torch.Size(6)
print(net.conv1.bias.grad)

loss.backward() # differentiation

print('conv1.weight.grad after backward')
print(net.conv1.weight.grad)

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)


conv1.weight.grad before backward
tensor([[[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]])
conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.weight.grad after backward
tensor([[[[-6.2256e-03, -7.0671e-03,  1.9850e-03],
          [-2.4205e-02,  1.1469e-02, -1.1738e-02],
          [-4.7517e-03, -2.7548e-03,  2.4390e-03]]],


        [[[ 1.0485e-02, -9.4935e-03,  1.3113e-02],
          [ 2.1902e-02,  2.7708e-02, -4.0821e-03],
          [-3.1640e-03, -7.2389e-03, -1.0532e-03]]],


        [[[-1.2317e-03, -1.0144e-02, -8.5990e-05],
          [-6.5920e-03,  3.4799e-03, -8.9887e-04

In [15]:
# update weights

In [16]:
lr = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * lr)

In [17]:
# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

In [18]:
print(net.conv1.weight) # updated weight of conv1
print(net.conv1.bias) # updated bias of conv1

Parameter containing:
tensor([[[[ 0.0539,  0.3103, -0.2617],
          [-0.0726,  0.2297, -0.1196],
          [ 0.2726,  0.0332, -0.0647]]],


        [[[-0.1699, -0.0201,  0.1948],
          [ 0.0737,  0.0441, -0.1629],
          [ 0.3165,  0.3194,  0.1159]]],


        [[[ 0.0052,  0.3012, -0.2740],
          [ 0.1161,  0.2156, -0.0856],
          [ 0.0592,  0.1318,  0.2348]]],


        [[[-0.3293,  0.0444, -0.1716],
          [-0.3066,  0.0106,  0.1090],
          [-0.3205,  0.0039, -0.0434]]],


        [[[ 0.3089, -0.0144,  0.3036],
          [-0.1949,  0.0770,  0.2153],
          [-0.2601,  0.1125,  0.1511]]],


        [[[-0.0114, -0.2684, -0.2095],
          [ 0.0441,  0.1571, -0.2683],
          [ 0.0199, -0.0286,  0.2261]]]], requires_grad=True)
Parameter containing:
tensor([ 0.3053,  0.1891,  0.0585, -0.1480, -0.1886, -0.2541],
       requires_grad=True)
